In [2]:
%time
import pandas as pd
fi=pd.read_csv('./searches.csv', sep="^", low_memory=False, dtype=str, chunksize=1000000)

all_chunks=pd.DataFrame()
for i,chunk in enumerate(fi):
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print(i, len(all_chunks))
    
all_chunks.to_csv('searches.no_dup.csv', sep="^", index=False)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 4.77 µs
0 358999
1 359003
2 359003
3 359003
4 359003
5 359003
6 359003
7 359003
8 359003
9 359003
10 359003
11 359003
12 359003
13 359003
14 359003
15 359003
16 359003
17 359003
18 359003
19 359003
20 359004


In [3]:
%time
import pandas as pd
fi=pd.read_csv('./bookings.csv.bz2', sep="^", low_memory=False, dtype=str, chunksize=1000000)

all_chunks=pd.DataFrame()
for i,chunk in enumerate(fi):
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print(i, len(all_chunks))
    
all_chunks.to_csv('bookings.no_dup.csv', sep="^", index=False)


CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 21.7 µs
0 1000000
1 1000000
2 1000000
3 1000000
4 1000000
5 1000003
6 1000003
7 1000003
8 1000003
9 1000003
10 1000003


In [4]:
#Action Plan
    #1. Seleccionar las columnas de interes booking ('dep_port','arr_port','cre_date           ')
    #2. Arreglar espacios en aeropuertos de booking
    #3. Eliminar los duplicados de bookings
    #4. Poner fecha en mismo formato (str)
    #4. Añadir a la tabla una columna de todo 1
    #5. Search left join con booking
    #6. Fillna(0) para eliminar los nulos
    #7. Save file

In [6]:
b=pd.read_csv("./bookings.no_dup.csv", usecols=['dep_port', 'arr_port','cre_date           '], sep="^")
s=pd.read_csv("./searches.no_dup.csv", sep="^", low_memory=False)

In [8]:
b.columns=b.columns.str.strip()
b.dep_port=b.dep_port.str.strip()
b.arr_port=b.arr_port.str.strip()

In [9]:
b.dep_port[0], b.arr_port[0]

('ZRH', 'LHR')

In [10]:
b.drop_duplicates(inplace=True)

In [11]:
b.cre_date[0]

'2013-02-22 00:00:00'

In [12]:
s.Date[0]

'2013-01-01'

In [13]:
b.cre_date=b.cre_date.str[:10]

In [15]:
b.cre_date[0]

'2013-02-22'

In [16]:
b['booked']=1

In [17]:
s_b=s.merge(b, how='left', left_on=['Date', 'Origin','Destination'],
           right_on=['cre_date','dep_port','arr_port'])

In [18]:
len(s), len(s_b)

(359004, 359004)

In [19]:
s_b.shape

(359004, 49)

In [20]:
s_b.drop(['cre_date','dep_port','arr_port'],axis=1, inplace=True)

In [21]:
s_b.shape

(359004, 46)

In [22]:
s_b[s_b['booked']==1].head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,booked
27,2013-01-01,18:33:28,CCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,1.0
40,2013-01-01,06:36:57,FFP,86f167b84e77346849f9439ae87c02a6,SA,DMM,MNL,1.0,2.0,DMM,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,1.0
59,2013-01-01,07:00:38,FQD,e8741eaf2fa2f71f931475d18fa72096,US,ATL,MIA,0.0,1.0,ATL,...,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SEA,1.0
134,2013-01-01,23:30:44,MPT,5215502d9524c3183f3839b0d9a5e4f9,AU,MEL,SYD,0.0,1.0,MEL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SYD,1.0
172,2013-01-01,18:34:27,MPT,fd4afff0035bec8f8e305d38804c33f6,IN,BOM,JED,1.0,2.0,BOM,...,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,DEL,1.0
